In [ ]:
from scapy.all import rdpcap

In [ ]:
pcap = rdpcap(
    "../data/blog_show_data/2023-01-23-Google-ad-to-possible-TA505-activity.pcap"
)
type(pcap)

In [ ]:
from scapy.all import *
import urllib.parse


def parse_http_payload(packet):
    # check if there is payload in packet
    if packet.haslayer(TCP) and packet.haslayer(Raw):
        try:
            payload = str(packet[Raw].load, "utf-8")

            # is it http?
            if "HTTP/" in payload:
                print(payload)
                body = None
                if "\r\n\r\n" in payload:
                    body = payload.split("\r\n\r\n")[1]
                else:
                    body = payload.split("<!DOCTYPE html>")[1]
                http_request = payload.split("\r\n")
                try:
                    # this generates a ValueError if the http request does not have the proper structure
                    method, path, protocol = http_request[0].split(" ")
                    print(method, path, protocol)
                    headers = {}
                    for header in http_request[1:]:
                        if header:
                            header_name, header_value = header.split(":", maxsplit=1)
                            headers[header_name] = header_value.strip()
                    parsed_path = urllib.parse.urlparse(path)
                    return {
                        "timestamp": str(packet.time),
                        "source_ip": packet[IP].src,
                        "dest_ip": packet[IP].dst,
                        "source_port": packet[TCP].sport,
                        "dest_port": packet[TCP].dport,
                        "method": method,
                        "path": parsed_path.path,
                        "query_string": parsed_path.query,
                        "protocol": protocol,
                        "headers": headers,
                        "body": body,
                    }
                # else:
                except:
                    # http header may be non-standard, may have no method or may be  error code like 302
                    return {
                        "timestamp": str(packet.time),
                        "source_ip": packet[IP].src,
                        "dest_ip": packet[IP].dst,
                        "source_port": packet[TCP].sport,
                        "dest_port": packet[TCP].dport,
                        "body": body,
                    }
        except:
            pass


# read a malicious packet capture downloaded from https://malware-traffic-analysis.net/
pcap = rdpcap(
    "../data/blog_show_data/2023-01-23-Google-ad-to-possible-TA505-activity.pcap"
)

for packet in pcap:
    parsed_payload = parse_http_payload(packet)
    if parsed_payload != None:
        print(parsed_payload)